<a href="https://colab.research.google.com/github/wtsyang/UserIntentPrediction/blob/BERT/BERT/LSTM-multiPrediction_addPenality_ReduceParameters_lrschedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/UserIntentPrediction'

/content/drive/My Drive/UserIntentPrediction


In [38]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers  import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import text, sequence
from gensim.models import KeyedVectors
from sklearn.model_selection  import train_test_split
import pickle
import sklearn
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import smart_cond
from functools import partial
from sklearn.metrics import precision_recall_fscore_support
print('Tensorflow Version:',tf.__version__)

Tensorflow Version: 2.2.0-rc2


In [0]:
NUM_MODELS = 1
BATCH_SIZE = 32
LSTM_UNITS = 64
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
EPOCHS = 24
MAX_LEN = 1259
N_CHANNELS=768
N_CLASS=12

## Loading the dataset


In [0]:
Train=pd.read_csv('data/Train_Preprocessing.csv').reset_index(drop=True)
Valid=pd.read_csv('data/Valid_Preprocessing.csv').reset_index(drop=True)
Test=pd.read_csv('data/Test_Preprocessing.csv').reset_index(drop=True)

In [0]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, pdDataFrame, dbName, labels=['oQ', 'RQ', 'CQ', 'FD', 'FQ', 'IR', 'PA', 'PF', 'NF', 'GG', 'JK', 'O'],\
                 batch_size=BATCH_SIZE, dim=MAX_LEN, n_channels=N_CHANNELS,\
                 n_classes=N_CLASS, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = pdDataFrame
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.dbName=dbName
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = self.list_IDs.iloc[indexes,:]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp.reset_index(drop=True))

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim, self.n_channels))
        y = np.zeros((self.batch_size,self.n_classes), dtype=int)

        # Generate data
        for i in range(len(list_IDs_temp)):
            utterenceID=list_IDs_temp.loc[i,'id']
            diaglogID=list_IDs_temp.loc[i,'diaglogID']
            try:
              temp=np.load('BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
              X[i,0:temp.shape[0],:] =temp 
              del temp
            except:
              print('Faile to load the data: BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
            # Store sample
            # Store class
            y[i,:] = np.array(list_IDs_temp.iloc[i,0:12])
        Y=[]
        for i in range(self.n_classes):
          Y+=[y[:,i].reshape((self.batch_size,))]
        return X, Y

In [0]:
training_generator = DataGenerator(Train,'Train')
validation_generator = DataGenerator(Valid,'Valid')

In [0]:
classWeight_Dict={}
for i in range(N_CLASS):
  ratioTrue=np.sum(Train.iloc[:,i])/len(Train)
  classWeight_Dict['output'+str(i+1)]={0:1+1/((1-ratioTrue)/(ratioTrue)+1),1:1+(1-ratioTrue)/(ratioTrue)/((1-ratioTrue)/(ratioTrue)+1)}

In [0]:
classWeight_Dict['output4'][1]+=1
classWeight_Dict['output4'][0]+=1

In [10]:
classWeight_Dict

{'output1': {0: 1.2345571818407344, 1: 1.7654428181592656},
 'output10': {0: 1.0336144877201687, 1: 1.9663855122798313},
 'output11': {0: 1.0100471347060282, 1: 1.9899528652939718},
 'output12': {0: 1.0018605805011163, 1: 1.9981394194988837},
 'output2': {0: 1.0604068469362442, 1: 1.9395931530637558},
 'output3': {0: 1.074423220044654, 1: 1.925576779955346},
 'output4': {0: 2.247085090548251, 1: 2.752914909451749},
 'output5': {0: 1.0875713222525427, 1: 1.912428677747457},
 'output6': {0: 1.1071694368643017, 1: 1.8928305631356983},
 'output7': {0: 1.3979161498387498, 1: 1.6020838501612502},
 'output8': {0: 1.1070453981642272, 1: 1.8929546018357728},
 'output9': {0: 1.076779955346068, 1: 1.923220044653932}}

## Build the model


In [0]:
def binary_crossentropy(y_true, y_pred, weights,from_logits=False,label_smoothing=0):

    y_pred = ops.convert_to_tensor(y_pred)
    y_true = math_ops.cast(y_true, y_pred.dtype)
    label_smoothing = ops.convert_to_tensor(label_smoothing, dtype=K.floatx())
    def _smooth_labels():
      return y_true * (1.0 - label_smoothing) + 0.5 * label_smoothing
    y_true = smart_cond.smart_cond(label_smoothing,_smooth_labels, lambda: y_true)
    
    mask0 = tf.subtract(tf.constant(1.0, dtype=K.floatx()),y_true)
    mask0=tf.math.scalar_mul(tf.constant(weights[0], dtype=K.floatx()),mask0)
    mask1 =tf.math.scalar_mul(tf.constant(weights[1], dtype=K.floatx()),y_true)
    mask=tf.math.add(mask0,mask1)

    return K.mean(tf.math.multiply(K.binary_crossentropy(y_true, y_pred, from_logits=from_logits),mask), axis=-1)

In [0]:
def build_model():
    inputs = Input(shape=(MAX_LEN,N_CHANNELS))
    x = SpatialDropout1D(0.2)(inputs)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    RESULT=[]
    for i in range(N_CLASS):
      RESULT+=[Dense(1, activation='sigmoid',name='output'+str(i+1))(hidden)]
    LOSS={}
    for i in  range(N_CLASS):
      LOSS['output'+str(i+1)]=partial(binary_crossentropy, weights=classWeight_Dict['output'+str(i+1)])
      LOSS['output'+str(i+1)].__name__ = 'loss'+str(i+1)

    model = Model(inputs=inputs, outputs=RESULT)
    model.compile(loss=LOSS, optimizer='adam',metrics=[tf.keras.metrics.binary_accuracy])

    return model

In [13]:
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1259, 768)]  0                                            
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 1259, 768)    0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 1259, 128)    426496      spatial_dropout1d[0][0]          
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1259, 128)    98816       bidirectional[0][0]              
______________________________________________________________________________________________

In [0]:
callback=ReduceLROnPlateau(patience=1,min_lr=0.00001,factor=0.3)
Name='BERT/LSTM_addPenalty_64_lr.h5'
checkpointer = ModelCheckpoint(filepath=Name, verbose=1, save_best_only=True)

In [0]:
model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[checkpointer,callback])

Instructions for updating:
Please use Model.fit, which supports generators.
Faile to load the data: BERT/vector/Train_123379_12798.npy
Epoch 1/24
251/251 [==============================] - ETA: 0s - loss: 5.2986 - output1_loss: 0.5565 - output2_loss: 0.3381 - output3_loss: 0.4177 - output4_loss: 1.3617 - output5_loss: 0.4529 - output6_loss: 0.4783 - output7_loss: 0.5956 - output8_loss: 0.4166 - output9_loss: 0.4090 - output10_loss: 0.1453 - output11_loss: 0.0915 - output12_loss: 0.0355 - output1_binary_accuracy: 0.8186 - output2_binary_accuracy: 0.9360 - output3_binary_accuracy: 0.9241 - output4_binary_accuracy: 0.7522 - output5_binary_accuracy: 0.9094 - output6_binary_accuracy: 0.8909 - output7_binary_accuracy: 0.8225 - output8_binary_accuracy: 0.8947 - output9_binary_accuracy: 0.9219 - output10_binary_accuracy: 0.9622 - output11_binary_accuracy: 0.9856 - output12_binary_accuracy: 0.9945 
Epoch 00001: val_loss improved from inf to 4.19736, saving model to BERT/LSTM_addPenalty_64_lr.h5

## Prediction




In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1259, 768)]  0                                            
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 1259, 768)    0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 1259, 128)    426496      spatial_dropout1d[0][0]          
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1259, 128)    98816       bidirectional[0][0]              
______________________________________________________________________________________________

In [0]:
model.load_weights(Name)

In [0]:
class testDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, pdDataFrame, dbName, labels=['oQ', 'RQ', 'CQ', 'FD', 'FQ', 'IR', 'PA', 'PF', 'NF', 'GG', 'JK', 'O'],\
                 batch_size=25, dim=MAX_LEN, n_channels=N_CHANNELS,\
                 n_classes=N_CLASS, shuffle=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = pdDataFrame
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.dbName=dbName
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = self.list_IDs.iloc[indexes,:]

        # Generate data
        X = self.__data_generation(list_IDs_temp.reset_index(drop=True))

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim, self.n_channels))
        #y = np.zeros((self.batch_size,self.n_classes), dtype=int)

        # Generate data
        for i in range(len(list_IDs_temp)):
            utterenceID=list_IDs_temp.loc[i,'id']
            diaglogID=list_IDs_temp.loc[i,'diaglogID']
            try:
              temp=np.load('BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
              X[i,0:temp.shape[0],:] =temp 
              del temp
            except:
              print('Faile to load the data: BERT/vector/'+self.dbName+'_'+str(utterenceID)+'_'+str(diaglogID)+'.npy')
            # Store sample
            # Store class
            #y[i,:] = np.array(list_IDs_temp.iloc[i,0:12])

        return X

In [0]:
test_generator=testDataGenerator(Test,'Test')

In [19]:
prediction = model.predict_generator(test_generator)

Instructions for updating:
Please use Model.predict, which supports generators.
Faile to load the data: BERT/vector/Test_250089_25728.npy


In [0]:
Prediction=np.array(prediction)

In [21]:
IOU=0
for i in range(len(Test)):
  pred=Prediction[0:N_CLASS,i,0]
  labels=np.array(Test.iloc[i,0:N_CLASS])
  ioU=np.sum((pred>=0.5)&(labels==1))/np.sum( (pred>=0.5) | (labels==1))
  IOU+=ioU
IOU=IOU/len(Test)
print(IOU)

0.6471171171171172


In [22]:
ACC=0
for i in range(len(Test)):
  pred=Prediction[0:N_CLASS,i,0]
  labels=np.array(Test.iloc[i,0:N_CLASS])
  acc=np.sum((pred>=0.5)==labels)/12
  ACC+=acc
ACC=ACC/len(Test)
print(ACC)

0.9257657657657649


In [23]:
for i in range(N_CLASS):
  print(np.sum(Test.iloc[:,i]==1)/len(Test),np.sum((Prediction[i,:,0]>=0.5)==(Test.iloc[:,i]))/len(Test))

0.23135135135135135 0.92
0.05837837837837838 0.9437837837837838
0.08108108108108109 0.9091891891891892
0.24756756756756756 0.7718918918918919
0.0918918918918919 0.9091891891891892
0.11135135135135135 0.932972972972973
0.4043243243243243 0.9124324324324324
0.11891891891891893 0.9221621621621622
0.07243243243243243 0.9297297297297298
0.02918918918918919 0.9686486486486486
0.004324324324324324 0.9913513513513513
0.002162162162162162 0.9978378378378379


In [41]:
for i in range(N_CLASS):
  pred=Prediction[i,:,0]
  labels=np.array(Test.iloc[:,i])
  print(precision_recall_fscore_support(labels, pred>=0.5, average='binary',zero_division=1))

(0.8181818181818182, 0.8411214953271028, 0.8294930875576038, None)
(0.525, 0.3888888888888889, 0.4468085106382979, None)
(0.38461538461538464, 0.2, 0.2631578947368421, None)
(0.5529411764705883, 0.4104803493449782, 0.47117794486215536, None)
(0.509090909090909, 0.32941176470588235, 0.4, None)
(0.735632183908046, 0.6213592233009708, 0.6736842105263158, None)
(0.9103641456582633, 0.8689839572192514, 0.8891928864569083, None)
(0.6826923076923077, 0.6454545454545455, 0.663551401869159, None)
(0.5161290322580645, 0.47761194029850745, 0.49612403100775193, None)
(0.4772727272727273, 0.7777777777777778, 0.5915492957746479, None)
(0.16666666666666666, 0.25, 0.2, None)
(1.0, 0.0, 0.0, None)


In [49]:
# Precision
Precision=0
N=len(Test)
for i in range(len(Test)):
  pred=Prediction[0:N_CLASS,i,0]
  labels=np.array(Test.iloc[i,0:N_CLASS])
  if np.sum(pred>=0.5)==0:
    N-=1
    continue
  precision=np.sum(((pred>=0.5)&(labels==1)))/np.sum(pred>=0.5)
  Precision+=precision
Precision=Precision/N
print(Precision)

0.7673495754891102


In [58]:
# Precision
Precision=0
N=N_CLASS
for i in range(N_CLASS):
  pred=Prediction[i,:,0]
  labels=np.array(Test.iloc[:,i])
  if np.sum(pred>=0.5)==0:
    N-=1
    continue
  precision=np.sum(((pred>=0.5)&(labels==1)))/np.sum(pred>=0.5)
  Precision+=precision
Precision=Precision/N
print(Precision)

0.5707805774377068


In [50]:
Recall=0
N=len(Test)
for i in range(len(Test)):
  pred=Prediction[0:N_CLASS,i,0]
  labels=np.array(Test.iloc[i,0:N_CLASS])
  if np.sum(labels)==0:
    N-=1
    continue
  recall=np.sum(((pred>=0.5)&(labels==1)))/np.sum(labels)
  Recall+=recall
Recall=Recall/N
print(Precision)

0.7673495754891102


In [60]:
# Precision
Recall=0
N=N_CLASS
for i in range(N_CLASS):
  pred=Prediction[0:N_CLASS,i,0]
  labels=np.array(Test.iloc[i,0:N_CLASS])
  if np.sum(labels)==0:
    N-=1
    continue
  recall=np.sum(((pred>=0.5)&(labels==1)))/np.sum(labels)
  Recall+=recall
Recall=Recall/N
print(Recall)

0.7083333333333334


In [0]:
y_true=np.array(Test.iloc[:,0:N_CLASS])
y_pred=Prediction

In [0]:
def hamming_score(y_true, y_pred, toggle_output=False):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_pred.shape[1]):
        set_true = set( np.where(y_true[i,:])[0])
        set_pred = set( np.where(y_pred[:,i,0]>=0.5)[0])
        if toggle_output:
            print('set_true: {0}'.format([id2label[id] for id in set_true]), 'set_pred: {0}'.format([id2label[id] for id in set_pred]))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [88]:
hamming_score(y_true, y_pred, toggle_output=False)

0.6471171171171172

In [0]:
   
def f1(y_true, y_pred):
    correct_preds, total_correct, total_preds = 0., 0., 0.
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i,:])[0])
        set_pred = set( np.where(y_pred[:,i,0]>=0.5)[0])
        
        correct_preds += len(set_true & set_pred)
        total_preds += len(set_pred)
        total_correct += len(set_true)

    p = correct_preds / total_preds if correct_preds > 0 else 0
    r = correct_preds / total_correct if correct_preds > 0 else 0
    f1 = 2 * p * r / (p + r) if correct_preds > 0 else 0
    return p, r, f1

In [90]:
f1(y_true, y_pred)

(0.7195945945945946, 0.6339285714285714, 0.6740506329113924)